In [1]:
from confluent_kafka import Producer
from pyspark.sql.types import *
from pyspark.sql import Row
from pyspark.sql import SparkSession
from pyspark import SparkContext
from time import sleep

In [2]:
spark = SparkSession.builder.master("local[4]").getOrCreate()
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)

In [3]:
schema = StructType([StructField('track_uri', StringType(), False),StructField('track_name', StringType(), False)])
user_data = spark.read.format("csv").schema(schema).load("DataToStream.csv")
user_data_collection = user_data.rdd.collect()

In [15]:
p = Producer({'bootstrap.servers': 'localhost:9092'})

In [16]:
batch_size = 0
batch_id = 1
for row in user_data_collection:
    if row is None: 
        print("No track")
        continue
    if batch_size < 10:    
        batch_size += 1
        msg = " ".join([row.track_uri,",",row.track_name])
        p.produce("spotify-recommend_v1", key = str(batch_id), value = msg)
    else:
        batch_size = 0
        batch_id += 1
        sleep(0.5)
        
msg = " ".join("stop")        
p.produce("spotify-recommend1", key = str('0'), value = msg)